In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [2]:
# Load data and assign number of CPU cores
df = pd.read_csv('../data/creditcard.csv')
os.environ["LOKY_MAX_CPU_COUNT"] = "2"

In [3]:
# Scale the amount feature
scaler = StandardScaler()
df['Amount_scaled'] = scaler.fit_transform(df[['Amount']])
df.drop(['Time', 'Amount'], axis=1, inplace=True)

# Split into Features 'X' and 'y' for train-test split
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [5]:
from imblearn.over_sampling import SMOTE

# Improve class imblance with SMOTE on training data
sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

In [6]:
# Train Logistic Regression Model
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_sm, y_train_sm)
y_pred = lr.predict(X_test)
print("---- LogisticRegression ----")
print(classification_report(y_test, y_pred))

---- LogisticRegression ----
              precision    recall  f1-score   support

           0       1.00      0.97      0.99     56864
           1       0.06      0.92      0.11        98

    accuracy                           0.97     56962
   macro avg       0.53      0.95      0.55     56962
weighted avg       1.00      0.97      0.98     56962



In [7]:
# Train Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_sm, y_train_sm)
y_pred = rf.predict(X_test)
print("---- RandomForest ----")
print(classification_report(y_test, y_pred))

---- RandomForest ----
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.87      0.83      0.85        98

    accuracy                           1.00     56962
   macro avg       0.94      0.91      0.92     56962
weighted avg       1.00      1.00      1.00     56962



In [8]:
# Train XGBoost Model
xgb = XGBClassifier(eval_metric='logloss')
xgb.fit(X_train_sm, y_train_sm)
y_pred = xgb.predict(X_test)
print("---- XGBClassifier ----")
print(classification_report(y_test, y_pred))

---- XGBClassifier ----
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.69      0.87      0.77        98

    accuracy                           1.00     56962
   macro avg       0.84      0.93      0.88     56962
weighted avg       1.00      1.00      1.00     56962



- Logistic Regression had the highest recall but extremely low precision (0.06), meaning it labeled almost everything as fraud. While it catches most fraud cases, it also misclassifies a lot of legitimate transactions.
- Random Forest gave a strong overall result, with high scores in both precision and recall. It did a better job of catching fraud without over-flagging regular activity.
- XGBoost performed similarly, with slightly better recall than Random Forest but lower precision. 

Since this project deals with financial transactions, precision matters more. It’s more important to avoid flagging normal purchases than to catch every single fraud attempt. Based on that, I’ll be using the Random Forest model for the next phase.


In [9]:
import joblib

# Save the Random Forest trained AI model for application use
joblib.dump(rf, '../models/fraud_model.pkl')

['../models/fraud_model.pkl']